# Benchmark with Movielens dataset
The main purpose of this notebook is not to produce comprehensive benchmarking results on multiple datasets. Rather, it is intended to evaluate different recommender algorithms(SVD, LightGCN, Transformer and our algorithm) in this repository.

* Datasets
  * [Movielens 100K](https://grouplens.org/datasets/movielens/100k/).
  * [Movielens 1M](https://grouplens.org/datasets/movielens/1m/).

* Data split
  * TODO
  

* Evaluation metrics
  * Ranking metrics:
    * Precision@k.
    * Recall@k.
    * Normalized discounted cumulative gain@k (NDCG@k).
    * Mean-average-precision (MAP). 
  * Rating metrics:
    * Root mean squared error (RMSE).
    * Mean average error (MAE).
    * R squared.
    * Explained variance.

In [9]:
!pip install torch

In [10]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [11]:
import warnings
warnings.filterwarnings("ignore")
import logging
logging.basicConfig(level=logging.ERROR)

import numpy as np
import pandas as pd

from train_and_evaluate import svd_model_train_and_evaluate, lgcn_model_train_and_evaluate

In [12]:
def generate_summary(data, algo, k, rating_metrics, ranking_metrics):
    summary = {"Data": data, "Algo": algo, "K": k}
    if rating_metrics is None:
        rating_metrics = {
            "RMSE": np.nan,
            "MAE": np.nan,
            "R2": np.nan,
            "Explained Variance": np.nan,
        }
    if ranking_metrics is None:
        ranking_metrics = {
            "MAP": np.nan,
            "nDCG@k": np.nan,
            "Precision@k": np.nan,
            "Recall@k": np.nan,
        }
    summary.update(rating_metrics)
    summary.update(ranking_metrics)
    return summary

In [ ]:
def benchmark_recommenders():
    cols = ["Data", "Algo", "K", "RMSE", "MAE", "R2", "Explained Variance", "MAP", "nDCG@k", "Precision@k", "Recall@k"]
    df_results = pd.DataFrame(columns=cols)
    sizes = ["100k"]
    # algos=["svd", "lgcn"]
    algos=["lgcn"]
    models={"svd":svd_model_train_and_evaluate, "lgcn":lgcn_model_train_and_evaluate}
    for size in sizes:
        for algo in algos:
            ratings, rankings = models[algo](size)
            summary = generate_summary(size, algo, 10, ratings, rankings)
            df_results.loc[df_results.shape[0] + 1] = summary
    return df_results



In [15]:
df_results = benchmark_recommenders()
df_results

   user  item  prediction
0     1     1    4.018125
3     1     4    4.204126
5     1     6    3.188719
7     1     8    4.805475
9     1    10    4.200003
   user  item  prediction  rank
0   196   258    0.770406     1
1   196    50    0.766580     2
2   196   288    0.756307     3
3   196   294    0.756078     4
4   196   300    0.743543     5


KeyError: 'rank'